In [11]:
import numpy as np
import pandas as pd
from scipy import stats
from utils import get_data, save_data, get_test_preds, add_intervals_to_test, all_tests, save_param_values, get_race_info 
from plots import plots_for_race, plot_interval_checks
np.random.seed(2025)

race = "chi"
size1, size2 = 2000, 20000
train_yr, test_yr = [2021, 2022, 2023], [2024]

# save_data(race_list=["bos", "nyc", "chi"], size_train=size1, size_test=size2, train_lis=train_yr, test_lis=test_yr)
train, test = get_data(racename=race, size_train=size1, size_test=size2, train_lis=train_yr, test_lis=test_yr, save=False)
params = save_param_values(race)
get_race_info()

models, baseline = ["M1", "M2", "M3"], "BL"
test_preds = get_test_preds(test, race, baseline=baseline, full=False)
test_preds

,id,dist,curr_pace,total_pace,finish,age,gender,year,male,malexage,alpha,lvl,BL,M1,M2,M3
23464,138676,5K,2.981515,2.981515,2.907794,39,M,2024,1,39,1,1,-5.979950,8.912357,8.896137,11.735330
41746,156958,5K,2.364066,2.364066,2.264895,44,W,2024,0,0,1,1,-13.025250,14.108817,14.077674,10.399526
8633,123845,5K,3.750938,3.750938,3.492385,49,M,2024,1,49,1,1,-13.880217,-5.990898,-5.998956,-4.492944
3865,119077,5K,3.918495,3.918495,3.835909,39,M,2024,1,39,1,1,-3.863933,3.076218,3.069238,3.885235
26977,142189,5K,2.932551,2.932551,2.773250,34,W,2024,0,0,1,1,-13.775083,1.785914,1.768898,-2.096194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388017,137374,40K,2.369668,2.952030,2.949049,59,M,2024,1,59,1,8,-0.240729,0.360740,1.326505,1.550054
417494,166851,40K,1.319958,1.684636,1.660763,24,W,2024,0,0,1,8,-6.000800,-4.870773,-4.014145,-4.847535
390499,139856,40K,2.530364,2.865124,2.860290,59,W,2024,0,0,1,8,-0.414835,0.206680,0.415610,0.464460
370866,120223,40K,3.548616,3.750938,3.727473,29,M,2024,1,29,1,8,-1.180217,-0.716522,-0.842914,-0.868173


In [12]:
test_list = [["BL", "M1"], ["BL", "M2"], ["BL", "M3"],["M1", "M2"], ["M1", "M3"], ["M2", "M3"]]
tbl2 = all_tests(test_preds, test_list, savename=f"analysis/tables/{race}_stattest.csv")
print(tbl2)

comp_model = "M2"
error_tbl = plots_for_race(test_preds, race, models, baseline, comp_model)
error_tbl

/Users/brandononyejekwe/Documents/Personal_Projects/Marathon_Predictor/utils.py:249: UserWarning: p-value floored: true value smaller than 0.001. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)
  value = stats.anderson_ksamp([arr1, arr2]).pvalue


            KS Wilcoxon      CVM      AD
BL-M1  <0.0001  <0.0001  <0.0001  0.0010
BL-M2  <0.0001  <0.0001  <0.0001  0.0010
BL-M3  <0.0001  <0.0001  <0.0001  0.0010
M1-M2  <0.0001  <0.0001  <0.0001  0.0010
M1-M3  <0.0001  <0.0001  <0.0001  0.0010
M2-M3   0.0172  <0.0001   0.0333  0.0527
File saved: analysis/plots/chi_error_bar.jpg
File saved: analysis/tables/chi_error.csv
File saved: analysis/plots/chi_error_groups.jpg
File saved: analysis/plots/chi_error_gender_age.jpg


,BL,M1,M2,M3,pcnt_M1,pcnt_M2,pcnt_M3
5K,20.678817,16.468657,16.466335,16.369257,0.203598,0.203710,0.208405
10K,18.328082,14.178194,13.630528,13.476796,0.226422,0.256304,0.264691
15K,16.808693,13.077648,12.387477,12.131352,0.221971,0.263032,0.278269
20K,15.742424,10.997974,9.634350,9.325690,0.301380,0.388001,0.407608
25K,13.350323,9.292951,7.511434,7.407809,0.303916,0.437359,0.445121
30K,9.230582,6.639586,4.889068,4.854046,0.280697,0.470340,0.474134
35K,5.450820,3.876190,2.860048,2.817827,0.288879,0.475300,0.483045
40K,1.148686,1.106537,0.861050,0.838025,0.036693,0.250405,0.270449
Overall MAE,12.604000,9.468000,8.549000,8.422000,NaN,NaN,NaN
Overall $R^2$,0.888000,0.936000,0.941000,0.941000,NaN,NaN,NaN


In [17]:
# train_full, test_full = get_data(racename=race, size_train=None, size_test=None, train_lis=train_yr, test_lis=test_yr, save=False)
# print(stats.ks_2samp(train["finish"], train_full["finish"]).pvalue)
# print(stats.anderson_ksamp([train["finish"], train_full["finish"]]).pvalue)
# print(stats.ks_2samp(test["finish"], test_full["finish"]).pvalue)
# print(stats.anderson_ksamp([test["finish"], test_full["finish"]]).pvalue)

# print(stats.ks_2samp(test["finish"], test_full["finish"]).pvalue)
# print(stats.ks_2samp(test["curr_pace"], test_full["curr_pace"]).pvalue)
# print(stats.ks_2samp(test["total_pace"], test_full["total_pace"]).pvalue)
# print(stats.ks_2samp(test["age"], test_full["age"]).pvalue)
# print(stats.ks_2samp(test["male"], test_full["male"]).pvalue)

In [14]:
mpreds2 = get_test_preds(test, race, baseline=baseline, full=True)
intervals_tbl = add_intervals_to_test(test_preds, mpreds2, models)
i_check, i_sizes = plot_interval_checks(intervals_tbl, models, save_name=race)

File saved: analysis/plots/chi_intervals
analysis/tables/chi_intsizes.csv
analysis/tables/chi_intcheck.csv


In [15]:
import pandas as pd
params = pd.read_csv(f"stan_results/model3/params_{race}.csv", index_col="Unnamed: 0")
params.describe().T[["mean", "std", "25%", "50%", "75%"]].round(4)

,mean,std,25%,50%,75%
beta.1.1,-0.2518,0.1002,-0.3197,-0.2508,-0.1819
beta.2.1,-0.1490,0.0943,-0.2122,-0.1474,-0.0842
beta.3.1,-0.2920,0.0657,-0.3381,-0.2920,-0.2462
beta.4.1,-0.0452,0.0517,-0.0789,-0.0449,-0.0109
beta.5.1,0.0529,0.0454,0.0226,0.0528,0.0838
beta.6.1,0.0316,0.0280,0.0133,0.0315,0.0504
beta.7.1,-0.0025,0.0183,-0.0151,-0.0023,0.0098
beta.8.1,0.0152,0.0043,0.0123,0.0153,0.0181
beta.1.2,0.5300,0.6903,0.0713,0.5260,0.9834
beta.2.2,0.3321,0.2107,0.1903,0.3323,0.4708
